In [1]:
import numpy as np 
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score 
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter

In [2]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [3]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [4]:
list_comp = ['input_width', 'input_size','nb_params_conv', 'nb_params_bn', 'nb_params_fc', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']

list_var = ['input_width', 'nb_params_bn', 'nb_params_fc','nb_conv_layers', 'nb_bn_layers', 'nb_fc_layers']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [5]:
y_train = train_dataset.pop('ACET')
X_train = train_dataset
X_train.tail()

,Model_name,input_shape,input_size,nb_params_conv,flops,nb_layers,sum_activations,weighted_sum_neurons
1328,basic_model_480_34,"(1, 480, 480, 3)",691200,33133184,79295508671,59,35042884,39594707232
1329,basic_model_256_13,"(1, 256, 256, 3)",196608,2093972,109798665152,25,135107384,54848514304
1330,basic_model_350_6,"(1, 350, 350, 3)",367500,172904,42303275359,15,127277600,21112385000
1331,basic_model_64_21,"(1, 64, 64, 3)",12288,25503568,59776080802,38,10843232,29858337280
1332,basic_model_400_14,"(1, 400, 400, 3)",480000,8727042,74335462549,27,84037806,37114318592


In [6]:
Y_e1 = test_dataset1.pop('ACET')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('ACET')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('ACET')
X_e3 = test_dataset3

In [7]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [8]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [9]:
steps = [('scaler', StandardScaler()), ('MLP', MLPRegressor())]
pipeline = Pipeline(steps)

In [10]:
"""
# The grid search hyperparameters space
parameteres = {'MLP__hidden_layer_sizes':[(58,16,16), (56,16,16), (48,16,16), (32,16,16), (32,32,16), (72,16), (48,16), (56,16)], 
               'MLP__activation':['relu'], 
               'MLP__solver':['adam'], 
               'MLP__alpha':[0.08, , 0.02, 0.003, 0.04, 0.05, 0.06, 0.07, 0.00008, 0.00009, .5, .05, .005, .0005],
               'MLP__learning_rate' :['adaptive', 'constant'],
               'MLP__learning_rate_init' :[0.1, 0.3, 0.2],
               'MLP__max_iter' :[5000], 
               'MLP__validation_fraction' :[0.1, 0.2, 0.3],
               'MLP__n_iter_no_change' :[10, 20, 30],
               'MLP__early_stopping' :[True],
               'MLP__shuffle' :[True],
               'MLP__random_state' :[42],
               }
"""               
               

# The optimal hyperparamaters
parameteres = {'MLP__hidden_layer_sizes':[(48,16)], 
               'MLP__activation':['relu'], 
               'MLP__solver':['adam'], 
               'MLP__alpha':[0.08],
               'MLP__learning_rate' :['adaptive'],
               'MLP__learning_rate_init' :[0.3],
               'MLP__max_iter' :[5000], 
               'MLP__validation_fraction' :[0.2],
               'MLP__n_iter_no_change' :[10],
               'MLP__early_stopping' :[True],
               'MLP__shuffle' :[True],
               'MLP__random_state' :[42],
               'MLP__verbose' :[False]
               }


In [11]:
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=2)

In [12]:
grid.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('MLP', MLPRegressor())]),
             param_grid={'MLP__activation': ['relu'], 'MLP__alpha': [0.08],
                         'MLP__early_stopping': [True],
                         'MLP__hidden_layer_sizes': [(48, 16)],
                         'MLP__learning_rate': ['adaptive'],
                         'MLP__learning_rate_init': [0.3],
                         'MLP__max_iter': [5000], 'MLP__n_iter_no_change': [10],
                         'MLP__random_state': [42], 'MLP__shuffle': [True],
                         'MLP__solver': ['adam'],
                         'MLP__validation_fraction': [0.2],
                         'MLP__verbose': [False]})

In [13]:
print(grid.best_params_)

{'MLP__activation': 'relu', 'MLP__alpha': 0.08, 'MLP__early_stopping': True, 'MLP__hidden_layer_sizes': (48, 16), 'MLP__learning_rate': 'adaptive', 'MLP__learning_rate_init': 0.3, 'MLP__max_iter': 5000, 'MLP__n_iter_no_change': 10, 'MLP__random_state': 42, 'MLP__shuffle': True, 'MLP__solver': 'adam', 'MLP__validation_fraction': 0.2, 'MLP__verbose': False}


In [14]:
# Select the best estimator found by the gridSearchCV
best_pipe = grid.best_estimator_

In [15]:
# Train the best estimator with the best hyperparams found by the gridSearchCV on the entire training data
best_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('MLP',
                 MLPRegressor(alpha=0.08, early_stopping=True,
                              hidden_layer_sizes=(48, 16),
                              learning_rate='adaptive', learning_rate_init=0.3,
                              max_iter=5000, random_state=42,
                              validation_fraction=0.2))])

In [16]:
y_pred1 = best_pipe.predict(X_e1.values)
y_pred2 = best_pipe.predict(X_e2.values)
y_pred3 = best_pipe.predict(X_e3.values)

In [27]:
y_test = y_test1
y_pred = y_pred1
test_models = X_test_model_names1.values
test_input_shapes = X_test_input_shapes1.values

In [28]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD :', median_absolute_error(y_test, y_pred))

Explained variance :  0.9961069193343572
R2_value :  0.9961056860243456
Mean absolute error MAE : 34.86760395081786
Root Mean squared error RMSE : 54.66596149953746
Median absolute error MAD : 22.075240230114275


In [29]:
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) 
accuracy = 100 - np.mean(mape)
print('Mape:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

Mape: 10.98 %.
Accuracy: 89.02 %.


In [30]:
import csv
with open('./measured_vs_predicted_MLP_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'real_exec_time', 'predicted_exec_time', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])